In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
i=0
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if i<1:
            print(os.path.join(dirname, filename))
        else:
            pass
        i+=1

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
import pandas as pd
import numpy as np
import wfdb
import ast

In [ ]:
path = "/kaggle/input/ptbxl-original-dataset/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/"

In [ ]:
#Load Annotation data
Y =pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')

In [ ]:
Y.columns

In [ ]:
# Checkin meta data of a single patient
Y.iloc[0]

#### Meta data contains all informations about patient including the signal file names and patient_id. Using these informations, we can easily find signals for the specific patient_id.

In [ ]:
# Evaluate data on scp_codes column
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

In [ ]:
#load ecg data for patient Y.iloc[0]
# wfdb.rdsamp() returns actual signal data and data information 
test_data,meta_data = wfdb.rdsamp(path+Y.iloc[0].filename_lr)

In [ ]:
meta_data

In [ ]:
print("test_data contains data of {1} leads where each lead has {0} samples".format(*test_data.shape))

## Let's load data using the provided example script example_physionet.py

In [ ]:

def load_raw_data(df, sampling_rate, path):
    # Loading all data with signal and meta information
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    
    # Eliminating meta information. We are selecting only signal value of 12 leads 
    data = np.array([signal for signal, meta in data])
    return data


sampling_rate=100

# Load raw signal data
X = load_raw_data(Y, sampling_rate, path)

In [ ]:
X.shape

In [ ]:
print("X has total {0} records of {2} leads having {1} samples in each lead".format(*X.shape))

In [ ]:
# Getting Superclass data
agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)

In [ ]:
# Let's check the columns
agg_df.columns

In [ ]:
# Checking Single Data
agg_df.iloc[0]

In [ ]:
# Only keep the rows that has agg_df.diagnostic == 1, this means diagnostinc report available
agg_df = agg_df[agg_df.diagnostic == 1]

## Adding diagnostic superclass to y data

In [ ]:
def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

# Apply diagnostic superclass
"""
If the dict keys in Y.scp_codes are available in agg_diff index, put them in diagnostic_superclass
 - This means scp_statements.csv contains diagnostic class in the first column
"""
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

## Plotting A signle ECG example

In [ ]:
# Let's look at shape of X and Y data
print(X.shape, Y.shape)
print("This means that each ecg_id in Y has corresponding indetical signal in X")

In [ ]:
# let's pick an arbitrary signal and it's information from Y index_column
ecg_record = 1239
Lead_II_data = X[ecg_record][1] # Index of Lead. It is available in meta_data information
ecg_info = Y.iloc[ecg_record]
pd.DataFrame(Lead_II_data).plot()
print(ecg_info)